In [1]:
!wget -q https://dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j
import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"
# Start a SparkSession
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df=spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",",  header=True)
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")


In [5]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [6]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query = """
SELECT YEAR (date) AS year, bedrooms, ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR(date), bedrooms
ORDER BY YEAR(date)

"""

spark.sql(query).show()


+----+--------+-------------+
|year|bedrooms|average_price|
+----+--------+-------------+
|2019|       4|     300263.7|
|2020|       4|    298353.78|
|2021|       4|    301819.44|
|2022|       4|    296363.88|
+----+--------+-------------+



In [7]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query = """
SELECT date_built,bedrooms, bathrooms, ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built,bedrooms,bathrooms
ORDER BY date_built

"""

spark.sql(query).show()


+----------+--------+---------+-------------+
|date_built|bedrooms|bathrooms|average_price|
+----------+--------+---------+-------------+
|      2010|       3|        3|    292859.62|
|      2011|       3|        3|    291117.47|
|      2012|       3|        3|    293683.19|
|      2013|       3|        3|    295962.27|
|      2014|       3|        3|    290852.27|
|      2015|       3|        3|     288770.3|
|      2016|       3|        3|    290555.07|
|      2017|       3|        3|    292676.79|
+----------+--------+---------+-------------+



In [8]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query = """
SELECT date_built,
       bedrooms,
       bathrooms,
       floors,
       ROUND(AVG(sqft_living), 2) AS sqft_living,
       ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >=2000
GROUP BY date_built, bedrooms, bathrooms, floors
ORDER BY date_built desc

"""
spark.sql(query).show()


+----------+--------+---------+------+-----------+-------------+
|date_built|bedrooms|bathrooms|floors|sqft_living|average_price|
+----------+--------+---------+------+-----------+-------------+
|      2017|       3|        3|     2|    2274.01|    280317.58|
|      2016|       3|        3|     2|    2274.17|     293965.1|
|      2015|       3|        3|     2|    2308.93|    297609.97|
|      2014|       3|        3|     2|     2285.6|    298264.72|
|      2013|       3|        3|     2|    2296.37|    303676.79|
|      2012|       3|        3|     2|    2334.57|    307539.97|
|      2011|       3|        3|     2|     2264.1|    276553.81|
|      2010|       3|        3|     2|    2264.46|    285010.22|
+----------+--------+---------+------+-----------+-------------+



In [9]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
query="""
SELECT  view, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE price >= 350000
GROUP BY view
"""
spark.sql(query).show()



print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|   7| 403005.77|
|  51| 788128.21|
|  15|  404673.3|
|  54| 798684.82|
|  11| 399548.12|
|  29| 397771.65|
|  69| 750537.94|
|  42|  396964.5|
|  87| 1072285.2|
|  73| 752861.18|
|  64| 767036.67|
|   3|  398867.6|
|  30|  397862.0|
|  34| 401419.75|
|  59|  791453.0|
|   8| 398592.71|
|  28| 402124.62|
|  22| 402022.68|
|  85|1056336.74|
|  16| 399586.53|
+----+----------+
only showing top 20 rows

--- 0.9588758945465088 seconds ---


In [10]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [11]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [12]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

query="""
SELECT  view, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE price >= 350000
GROUP BY view
"""
spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
|   7| 403005.77|
|  51| 788128.21|
|  15|  404673.3|
|  54| 798684.82|
|  11| 399548.12|
|  29| 397771.65|
|  69| 750537.94|
|  42|  396964.5|
|  87| 1072285.2|
|  73| 752861.18|
|  64| 767036.67|
|   3|  398867.6|
|  30|  397862.0|
|  34| 401419.75|
|  59|  791453.0|
|   8| 398592.71|
|  28| 402124.62|
|  22| 402022.68|
|  85|1056336.74|
|  16| 399586.53|
+----+----------+
only showing top 20 rows

--- 0.5310254096984863 seconds ---


RUNTIME of cached is 0.531 seconeds which is 0.427 seconeds faster then original runtime 0.958 seconed.

In [19]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("home_parquet")


In [20]:
# 11. Read the parquet formatted data.
parquet_df=spark.read.parquet("home_parquet")


In [21]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("home_sale_parquet")

In [22]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

query="""
SELECT  view, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE price >= 350000
GROUP BY view
"""
spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|   7| 403005.77|
|  51| 788128.21|
|  15|  404673.3|
|  54| 798684.82|
|  11| 399548.12|
|  29| 397771.65|
|  69| 750537.94|
|  42|  396964.5|
|  87| 1072285.2|
|  73| 752861.18|
|  64| 767036.67|
|   3|  398867.6|
|  30|  397862.0|
|  34| 401419.75|
|  59|  791453.0|
|   8| 398592.71|
|  28| 402124.62|
|  22| 402022.68|
|  85|1056336.74|
|  16| 399586.53|
+----+----------+
only showing top 20 rows

--- 0.495070219039917 seconds ---


RUNTIME of parquet is 0.499 seconeds which is 0.032 seconeds slightly faster then cached runtime 0.531 seconed.

In [23]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [24]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False